In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predbert/pred_bert.csv


In [2]:
import pandas as pd
import numpy as np
import os
import requests
import re

!pip install transformers -q
!pip install wandb -q

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

!pip install SentencePiece -q

import torch.nn.functional as F
from transformers import  T5Tokenizer , T5ForConditionalGeneration
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

%pip install -q wandb
import wandb
wandb.login()

Note: you may need to restart the kernel to use updated packages.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
df = pd.read_csv('/kaggle/input/predbert/pred_bert.csv')

In [5]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df

,Article,Reference Summary,Generated Summary
0,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...,Fear shakes Mexico border city after violence ...
1,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a...",Indian-origin boy finds millions of years old ...
2,Representative ImageA 38-year-old Indian man h...,A 38-year-old Indian man has been charged with...,Representative ImageA 38-year-old Indian man h...
3,Residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi...",China coronavirus updates: Shanghai has extend...
4,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...,Israeli PM Benjamin Netanyahu flew to Saudi Ar...
...,...,...,...
4482,FILE - Tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...,FILE - Tanks storing treated radioactive water...
4483,Villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N...",At least 16 people were killed and hundreds ot...
4484,"Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar..."
4485,Thai BoysMost members of the Thai youth footba...,Most members of the Thai youth football team r...,One of the boys will not join the ceremony as ...


In [8]:
wandb.init(project="English_t5_pred_BERT")

config = wandb.config         
config.TRAIN_BATCH_SIZE = 2  
config.VALID_BATCH_SIZE = 2    
config.TRAIN_EPOCHS = 8       
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 5e-5    
config.SEED = 42               
config.MAX_LEN = 512
config.SUMMARY_LEN = 75 


torch.manual_seed(config.SEED) 
np.random.seed(config.SEED)
torch.backends.cudnn.deterministic = True


tokenizer = T5Tokenizer.from_pretrained("t5-base")

wandb: Currently logged in as: saliq7 (nitk-ai). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [9]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }


In [12]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=75,
                num_beams=4,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [13]:
df.drop(['Article'], axis = 1, inplace = True)
df.rename(columns = {'Generated Summary':'ctext', 'Reference Summary':'text'}, inplace = True)
df.ctext = 'summarize: ' + df.ctext

In [15]:
df=df[['ctext','text']]
df

,ctext,text
0,summarize: Fear shakes Mexico border city afte...,The attacks took place in several neighborhood...
1,summarize: Indian-origin boy finds millions of...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,summarize: Representative ImageA 38-year-old I...,A 38-year-old Indian man has been charged with...
3,summarize: China coronavirus updates: Shanghai...,"China on Apr 21 reported 2,119 locally transmi..."
4,summarize: Israeli PM Benjamin Netanyahu flew ...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,summarize: FILE - Tanks storing treated radioa...,A massive earthquake and tsunami in 2011 destr...
4483,summarize: At least 16 people were killed and ...,"Over 680 tourists from France, Thailand, the N..."
4484,"summarize: Seif al-Islam, the son and one-time...","Seif al-Islam, the son and one-time heir appar..."
4485,summarize: One of the boys will not join the c...,Most members of the Thai youth football team r...


In [16]:
val_dataset = df

In [18]:
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

# parameters fo dataloaders
train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

# Create  Dataloaders 
val_loader = DataLoader(val_set, **val_params)


model = T5ForConditionalGeneration.from_pretrained("saliq7/T5_English")
model = model.to(device)


optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
wandb.watch(model, log="all")


#for epoch in range(config.VAL_EPOCHS):
predictions, actuals = validate(1, tokenizer, model, device, val_loader)
final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
final_df.to_csv('pred_BERTABS_t5.csv')
print('Output Files generated for review')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Completed 1500
Completed 1600
Completed 1800
Completed 2000
Completed 2100
Completed 2200
Output Files generated for review


In [29]:
df1 = pd.read_csv('/kaggle/working/pred_BERTABS_t5.csv')
df1.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [30]:
df1

,Generated Text,Actual Text
0,fear has invaded the Mexican border city of Re...,The attacks took place in several neighborhood...
1,"Siddak Singh Jhamat, known as Sid, was using a...","Siddak Singh Jhamat, known as Sid, was using a..."
2,28-year-old Indian man has been charged with c...,A 38-year-old Indian man has been charged with...
3,"China on Thursday reported 2,119 locally trans...","China on Apr 21 reported 2,119 locally transmi..."
4,Israeli media reported Monday that Prime Minis...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,Japan’s nuclear regulator on Wednesday approve...,A massive earthquake and tsunami in 2011 destr...
4483,National Disaster Mitigation Agency had said m...,"Over 680 tourists from France, Thailand, the N..."
4484,son and one-time heir apparent of late Libyan ...,"Seif al-Islam, the son and one-time heir appar..."
4485,group will spend nine days living in a monaste...,Most members of the Thai youth football team r...


In [21]:
!pip install rouge -q

In [26]:
df = pd.read_csv('/kaggle/input/predbert/pred_bert.csv')

In [27]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.columns = ['Article','Reference','Bert']
df

,Article,Reference,Bert
0,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...,Fear shakes Mexico border city after violence ...
1,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a...",Indian-origin boy finds millions of years old ...
2,Representative ImageA 38-year-old Indian man h...,A 38-year-old Indian man has been charged with...,Representative ImageA 38-year-old Indian man h...
3,Residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi...",China coronavirus updates: Shanghai has extend...
4,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...,Israeli PM Benjamin Netanyahu flew to Saudi Ar...
...,...,...,...
4482,FILE - Tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...,FILE - Tanks storing treated radioactive water...
4483,Villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N...",At least 16 people were killed and hundreds ot...
4484,"Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar..."
4485,Thai BoysMost members of the Thai youth footba...,Most members of the Thai youth football team r...,One of the boys will not join the ceremony as ...


In [31]:
df['+Abs'] = df1['Generated Text']
df

,Article,Reference,Bert,+Abs
0,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...,Fear shakes Mexico border city after violence ...,fear has invaded the Mexican border city of Re...
1,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a...",Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,Representative ImageA 38-year-old Indian man h...,A 38-year-old Indian man has been charged with...,Representative ImageA 38-year-old Indian man h...,28-year-old Indian man has been charged with c...
3,Residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi...",China coronavirus updates: Shanghai has extend...,"China on Thursday reported 2,119 locally trans..."
4,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...
...,...,...,...,...
4482,FILE - Tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...,FILE - Tanks storing treated radioactive water...,Japan’s nuclear regulator on Wednesday approve...
4483,Villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N...",At least 16 people were killed and hundreds ot...,National Disaster Mitigation Agency had said m...
4484,"Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar...",son and one-time heir apparent of late Libyan ...
4485,Thai BoysMost members of the Thai youth footba...,Most members of the Thai youth football team r...,One of the boys will not join the ceremony as ...,group will spend nine days living in a monaste...


In [32]:
df.to_csv('+abs.csv')

In [33]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(df['+Abs'], df['Reference'], avg = True)
RougeScore = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
RougeScore

,rouge-1,rouge-2,rouge-l
recall,0.398008,0.296512,0.370614
precision,0.476834,0.358362,0.443285
f-measure,0.426020,0.317917,0.396542
